#### El Parque Nacional Guatopo
**Autor:** Javier Martínez

**Tutor:** Isabel LLatas

Con GEE es posible acceder a una colección de imágenes (o a una imágen) según el catálogo de productos disponibles en [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/).

Para el siguiente ejemplo se ha seleccionado al *Parque Nacional Guatopo*. Por lo que;

1. Se importan los módulos necesarios.
2. Se da inicio a la API.
3. Se consulta la colección de imágenes [*WCMC/WDPA/current/polygons*](https://developers.google.com/earth-engine/datasets/catalog/WCMC_WDPA_current_polygons), seleccionando la imágen *Guatopo*.
4. Se crea el mapa del *Parque Nacional Guatopo*.

 
Se recomienda usar [Jupyter nbview](https://nbviewer.jupyter.org/) para la visualización y estudio del *Notebook*.


In [9]:
# Módulos
import ee
import folium
import geehydro

In [10]:
# Inicio de API
ee.Initialize()

In [11]:
polygonCollection = 'WCMC/WDPA/current/polygons' # Colección
polygonName = 'Guatopo' # Imágen
# Consultando la colección y seleccionando la  imagen Guatopo
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))

Una vez tomada la imágen se procede a crear un mapa con la ayuda de *folium*. Se ha seleccionado la localización $10°30'59.7"N 66°51'36.3"W$ para el inicio del mapa con un zoom de 9, además se incorpora el control de escala y el control de layers. 

In [12]:
# Mapa
point = [10.5165848,-66.8600685] # Centro del mapa.
Map = folium.Map(location=point,zoom_start=9,control_scale =True,width='100%',height='100%')
Map.addLayer(polygon,name=polygonName) # Agregando polígono
Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map

Para Guardar el mapa en formato .HTML;

> Map.save('mapa_guatopo.html')

In [13]:
# pixelLonLat returns an image with each pixel labeled with longitude and
# latitude values.
lonLat = ee.Image.pixelLonLat()

# Select the longitude and latitude bands, multiply by a large number then
# truncate them to integers.
lonGrid = lonLat\
            .select('longitude')\
            .multiply(10000000)\
            .toInt()

latGrid = lonLat\
            .select('latitude')\
            .multiply(10000000)\
            .toInt()

In [14]:
# To produce the grid, multiply the latitude and longitude images and then use
# reduce to vectors at the 10km resolution to group the grid into vectors.
grid = lonGrid\
            .multiply(latGrid)\
            .reduceToVectors({
                'geometry': polygon, # This is undefined until you draw a geometry.
                'scale': 10000,
                'geometryType': 'polygon',
            })

Map.addLayer(grid)
Map

EEException: Invalid argument for ee.Reducer(): ({'geometry': <ee.featurecollection.FeatureCollection object at 0x7f4af56d8fa0>, 'scale': 10000, 'geometryType': 'polygon'},).  Must be a ComputedObject.